# Tutorial of NvDocker interfaces

In [1]:
import socket
from dsd.sys.docker import NvDocker

try:
    socket.gethostbyname('dockerhost')
    base_url = 'http://dockerhost:3476'
except socket.error:
    base_url = 'http://localhost:3476'
    
nvd = NvDocker(base_url=base_url)

# GPU global informations

In [2]:
globalInfo = nvd.gpuGlobalInfo()

print 'Driver Version:', globalInfo['DriverVersion']
print 'CUDA Version:', globalInfo['CudaVersion']

Driver Version: 352.39
CUDA Version: 7.5


# GPU status

In [3]:
gpuInfo = nvd.gpuInfo()

for gpu in gpuInfo:
    print '#' * 20
    print 'Model:', gpu['Model']
    print 'Path:', gpu['Path']
    print 'Memory:', gpu['MemoryUsed'], '/', gpu['MemoryTotal'], 'MB'
    
    print 'Utilization GPU:', gpu['UtilizationGpu']
    print 'Utilization Memory:', gpu['UtilizationMemory']
    print 'Temperature:', gpu['Temperature']
    
    for p in gpu['Processes']:
        print ' '*8, 'PID:', p['PID'], '| Name:', p['Name'], '| MemoryUsed:', p['MemoryUsed'], 'MB'

####################
Model: Tesla K40m
Path: /dev/nvidia0
Memory: 1446 / 11519 MB
Utilization GPU: 80
Utilization Memory: 36
Temperature: 47
         PID: 20401 | Name: /usr/bin/python | MemoryUsed: 73 MB
         PID: 30093 | Name: python | MemoryUsed: 1347 MB
####################
Model: Tesla K40m
Path: /dev/nvidia1
Memory: 1510 / 11519 MB
Utilization GPU: 81
Utilization Memory: 37
Temperature: 51
         PID: 30093 | Name: python | MemoryUsed: 137 MB
         PID: 31724 | Name: python | MemoryUsed: 1347 MB


# Generate Docker CLI parameter

In [4]:
params = nvd.cliParams()
print params
params = nvd.cliParams([0, 1])
print params
params = nvd.cliParams([1])
print params
params = nvd.cliParams('all')
print params

{'VolumeDriver': '', 'Devices': [], 'Volumes': []}
{'volumeDriver': '', 'devices': [], 'volumes': []}
{u'VolumeDriver': u'nvidia-docker', u'Devices': [u'/dev/nvidia0', u'/dev/nvidia1', u'/dev/nvidiactl', u'/dev/nvidia-uvm'], u'Volumes': [u'nvidia_driver_352.39:/usr/local/nvidia:ro']}
{'volumeDriver': u'nvidia-docker', 'devices': [HCP(h=u'/dev/nvidia0', c=None, p=None), HCP(h=u'/dev/nvidia1', c=None, p=None), HCP(h=u'/dev/nvidiactl', c=None, p=None), HCP(h=u'/dev/nvidia-uvm', c=None, p=None)], 'volumes': [HCP(h=u'nvidia_driver_352.39', c=u'/usr/local/nvidia', p=u'ro')]}
{u'VolumeDriver': u'nvidia-docker', u'Devices': [u'/dev/nvidia1', u'/dev/nvidiactl', u'/dev/nvidia-uvm'], u'Volumes': [u'nvidia_driver_352.39:/usr/local/nvidia:ro']}
{'volumeDriver': u'nvidia-docker', 'devices': [HCP(h=u'/dev/nvidia1', c=None, p=None), HCP(h=u'/dev/nvidiactl', c=None, p=None), HCP(h=u'/dev/nvidia-uvm', c=None, p=None)], 'volumes': [HCP(h=u'nvidia_driver_352.39', c=u'/usr/local/nvidia', p=u'ro')]}
{u'Volu

# Invoke a GPU enabled container
## 1. get control objects

In [1]:
from dsd.sys.docker import PyDocker, HC, HCP
from dsd.sys.docker import NvDocker

import socket

# get docker object
try:
    socket.gethostbyname('dockerhost')
    base_url = 'http://dockerhost:4243'
except socket.error:
    base_url = 'unix:///var/run/docker.sock'
    
docker = PyDocker(base_url)

try:
    socket.gethostbyname('dockerhost')
    base_url = 'http://dockerhost:3476'
except socket.error:
    base_url = 'http://localhost:3476'

nvd = NvDocker(base_url=base_url)

## 2. build GPU image

In [2]:
imageTag = 'buildtest:gpu'
response = docker.build(tag=imageTag, dockerfilePath='../docker/test/gpu')
if response:
    for line in response:
        print line
else:
    print 'build failed'

# For Test
FROM dsdgroup/jupyter:gpu
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999

{"stream":"Step 1 : FROM dsdgroup/jupyter:gpu\n"}

{"stream":" ---\u003e 1d0c98845267\n"}

{"stream":"Step 2 : MAINTAINER zy \u003czy3381@gmail.com\u003e\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e 574da3156f96\n"}

{"stream":"Step 3 : EXPOSE 9999\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e 4d3b7d2c17d2\n"}

{"stream":"Successfully built 4d3b7d2c17d2\n"}




## 3. Run container by constructed parameters

In [11]:
params = nvd.cliParams(dev=[1])
params['image'] = 'buildtest:gpu'
params['detach'] = True
params['stdin_open'] = True
params['tty'] = True
params['command'] = None #'nvidia-smi'
params['name'] = 'container_test_gpu'
params['user'] = None
params['ports'] = [HC(1234, 8888), HC(5678, 7777), HC(c=9999)]


print params

#container = docker.create(**params) # WHAT'S WRONG IN THIS LINE?
container = docker.create(image=params['image'], 
           detach=params['detach'], 
           stdin_open=params['stdin_open'], 
           tty=params['tty'], 
           command=params['command'], 
           name=params['name'],
           user=params['user'],
           ports=params['ports'],
           volumes=params['volumes'],
           devices=params['devices']
           )

if container:
    print 'Container %s created!' % container.get('Id')
    
response = docker.start(container=container.get('Id'))
print 'start: ', response

# IT SEEMS devices are OK,
# BUT volumes are not corrected mapped

{'tty': True, 'name': 'container_test_gpu', 'image': 'buildtest:gpu', 'stdin_open': True, 'devices': [HCP(h=u'/dev/nvidia1', c=None, p=None), HCP(h=u'/dev/nvidiactl', c=None, p=None), HCP(h=u'/dev/nvidia-uvm', c=None, p=None)], 'volume_driver': u'nvidia-docker', 'command': None, 'user': None, 'volumes': [HCP(h=u'nvidia_driver_352.39', c=u'/usr/local/nvidia', p=u'ro')], 'detach': True, 'ports': [HC(h=1234, c=8888), HC(h=5678, c=7777), HC(h=None, c=9999)]}
buildtest:gpu True True True None container_test_gpu None [HC(h=1234, c=8888), HC(h=5678, c=7777), HC(h=None, c=9999)] [HCP(h=u'/dev/nvidia1', c=None, p=None), HCP(h=u'/dev/nvidiactl', c=None, p=None), HCP(h=u'/dev/nvidia-uvm', c=None, p=None)] [HCP(h=u'nvidia_driver_352.39', c=u'/usr/local/nvidia', p=u'ro')] None
Container 37fc020dc777c239ffe24e263465c77b5cea96e8e0e5751f019d72851edf0d10 created!
start:  True


In [12]:
response = docker.stop(container=container.get('Id'), timeout=0)
print 'stop: ', response

response = docker.rm(container=container.get('Id'), )
print 'rm: ', response

stop:  True
rm:  True
